In [1]:
import json

In [2]:
import requests

In [3]:
from io import BytesIO

In [4]:
from PIL import Image

In [5]:
cams_json_file_loc = "./cams.json"
log_file_loc = "./pics.log"
pic_folder = "./pics/"

In [6]:
from datetime import datetime as dt

In [7]:
def get_pic(cams_json_file_loc, log_file_loc, pic_folder):
    # get all cams
    with open(cams_json_file_loc, "r") as f:
        cams = json.load(f)
        
    cam_ids = [cam["cam_id"] for cam in cams]
    
    # read the log to see which cam was last pulled
    curr_cam_id = cam_ids[0]
    with open(log_file_loc, "r") as f:
        lines = f.readlines()
        if len(lines) > 0:
            last_line = lines[-1]
            last_cam_id = int(last_line.split("\t")[0].split(" ")[1])
            last_cam_id_ind = [i for i, cam_id in enumerate(cam_ids) if cam_id == last_cam_id][0]
            curr_cam_id_ind = last_cam_id_ind + 1 if last_cam_id_ind + 1 < len(cam_ids) else 0
            curr_cam_id = cam_ids[curr_cam_id_ind]
    
    curr_cam = cams[curr_cam_id]
    
    # get the image
    pull_dt = dt.now().strftime("%m%d%y_%H%M%S")
    img_addr = curr_cam["img_addr"]
    raw_img = requests.get(img_addr).content
    img = Image.open(BytesIO(raw_img))
    
    # save the image
    img_filename = "cam_id_{}_{}.png".format(curr_cam_id, pull_dt)
    img.save("{}{}".format(pic_folder, img_filename))
    
    # update the log
    with open(log_file_loc, "a") as f:
        f.write("cam_id: {} pulled: {} to: {}{}\n".format(curr_cam_id, pull_dt, pic_folder, img_filename))

In [10]:
get_pic(cams_json_file_loc = cams_json_file_loc, log_file_loc = log_file_loc, pic_folder = pic_folder)